In [1]:
#normal python stuff
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

In [2]:
#import local helper
import helpers

In [3]:
#local cadcad stuff

os.chdir("/Users/zargham/Documents/GitHub/cadcad-ri")
from cadcad.spaces import space
from cadcad.dynamics import block
from cadcad.spaces import EmptySpace, Bit, Real
from cadcad.points import Point
from cadcad.systems import Experiment

In [4]:
spaceType = type(EmptySpace)

In [5]:
print(spaceType)

<class 'cadcad.spaces.Space'>


In [6]:
## let's try something weird
## we will make a space containing a space

@space
class ContributionSchema:
    schema: spaceType

In [7]:
print(ContributionSchema)

Space ContributionSchema has dimensions {'schema': 'Space'}


In [8]:
@space
class InitSchema:
    a: int
    b: float
    c: str

In [9]:
@block
def randomSchemaChange(domain:Point[ContributionSchema])->Point[ContributionSchema]:
    input = deepcopy(domain)
    rng = np.random.randn()

    if rng <-1.0:
        #drop a dimension
        options = list(input.data['schema'].dimensions().keys())
        print(options)
        dk = np.random.choice(options)
        NewSchema = helpers.drop_dimension(input.data['schema'], dk)
        
        return Point(ContributionSchema, {'schema':NewSchema})

    elif rng >-1.0:
        #add a dimension
        nk = np.random.choice(helpers.WORDS)
        NewSchema = Bit.rename_dims({'bit':nk}) + input.data['schema']

        return Point(ContributionSchema, {'schema':NewSchema})
    
    else:
        return input
    #else:
        #nest a dimension


In [10]:
CS2 = ContributionSchema**2

In [11]:
print(CS2)

Space 2-ContributionSchema has dimensions {'contributionschema_0': 'ContributionSchema', 'contributionschema_1': 'ContributionSchema'}


In [12]:
@block
def schemaDistance(domain:Point[CS2])->Point[Real]:
    schema0 = domain['contributionschema_0'].data['schema'].unroll_schema()
    schema1 = domain['contributionschema_1'].data['schema'].unroll_schema()

    #schema0 and schema1 are nested dictionaries
    #include a distance measure for nested dictionaries here
    value = 0.0

    return Point(Real, value)


In [13]:
schemaPt = Point(ContributionSchema, {'schema':InitSchema})

In [14]:
print(schemaPt)

Point in space ContributionSchema has data
{
    "schema": "Space InitSchema has dimensions {'a': 'int', 'b': 'float', 'c': 'str'}"
}



In [15]:
print(randomSchemaChange(schemaPt))

Point in space ContributionSchema has data
{
    "schema": "Space Bit+InitSchema has dimensions {'byebye': 'bool', 'a': 'int', 'b': 'float', 'c': 'str'}"
}



In [16]:
print(randomSchemaChange(randomSchemaChange(schemaPt)))

Point in space ContributionSchema has data
{
    "schema": "Space Bit+Bit+InitSchema has dimensions {'hi': 'bool', 'hi_1': 'bool', 'a': 'int', 'b': 'float', 'c': 'str'}"
}



In [17]:
experiment_params = {"iteration_n": 2, "steps": 10}
my_experiment = Experiment(schemaPt, experiment_params, (randomSchemaChange,))

results = my_experiment.run()

['bananas', 'byebye', 'a', 'b', 'c']
['bananas', 'bananas_1', 'byebye', 'b', 'c']
['byebye', 'byebye_1', 'bananas_1', 'byebye_1_1', 'b', 'c']
['ho', 'byebye', 'byebye_1', 'bananas_1', 'byebye_1_1', 'b']
['hi', 'bananas', 'a', 'b', 'c']
['apples', 'hi', 'bananas', 'a', 'c']


In [18]:
results

In [19]:
results[0].data

In [20]:
for r in results:
    for pt in r.data:
        print(pt)

Point in space ContributionSchema has data
{
    "schema": "Space Bit+InitSchema has dimensions {'byebye': 'bool', 'a': 'int', 'b': 'float', 'c': 'str'}"
}

Point in space ContributionSchema has data
{
    "schema": "Space Bit+Bit+InitSchema has dimensions {'bananas': 'bool', 'byebye': 'bool', 'b': 'float', 'c': 'str'}"
}

Point in space ContributionSchema has data
{
    "schema": "Space Bit+Bit+InitSchema has dimensions {'bananas': 'bool', 'byebye': 'bool', 'b': 'float', 'c': 'str'}"
}

Point in space ContributionSchema has data
{
    "schema": "Space Bit+Bit+Bit+InitSchema has dimensions {'bananas_1': 'bool', 'byebye': 'bool', 'b': 'float', 'c': 'str'}"
}

Point in space ContributionSchema has data
{
    "schema": "Space Bit+Bit+Bit+InitSchema has dimensions {'bananas_1': 'bool', 'byebye': 'bool', 'b': 'float', 'c': 'str'}"
}

Point in space ContributionSchema has data
{
    "schema": "Space Bit+Bit+Bit+Bit+InitSchema has dimensions {'byebye': 'bool', 'bananas_1': 'bool', 'byebye_1':